# brings new insights into current trends regarding localities, topics and events.

# important!: open neo4j, gather required libraries

In [3]:
import os
import csv
from neo4j import GraphDatabase, basic_auth

#currently neo4j is running on localhost
neo4jUrl = os.environ.get('NEO4J_URL',"bolt://localhost")
neo4jUser = os.environ.get('NEO4J_USER',"neo4j")
neo4jPass = os.environ.get('NEO4J_PASSWORD',"1")

driver = GraphDatabase.driver(neo4jUrl, auth=basic_auth(neo4jUser, neo4jPass))

graph = driver.session()

# parameters for Cypher-queries

In [21]:
# 0 = date; 1=month; 2=week
dateAggregationOptions = ["date(datetime({epochmillis:toInt(e.time)}))", "date.truncate('month',datetime({epochmillis:toInt(e.time)}))", "date.truncate('week',datetime({epochmillis:toInt(e.time)}), {dayOfWeek:2})"]
dateAggregation = dateAggregationOptions[1]
amountOfReturnValues = "5"


# Top Events (measured at RSVP-count)

In [24]:
def get_top_events(tx):
    for record in tx.run("match (e:Event) with "+dateAggregation+" as date, collect(e) as ec, toInt(e.yes_rsvp_count) as yes_rsvp_count order by yes_rsvp_count desc unwind ec as ecu match (ecu:Event) with  date, collect('{Yes_Rsvp_Amount: '+yes_rsvp_count+'| Event_Name: '+ ecu.name+'} ')[.."+amountOfReturnValues+"] as rsvps_for_events  return  date, rsvps_for_events order by date desc"):
        print(str(record["date"]) + " | " + str(record["rsvps_for_events"]) +"\n")

with driver.session() as session:
    session.read_transaction(get_top_events)

2019-09-01 | ['{Yes_Rsvp_Amount: 22| Event_Name: Cloud Native} ', '{Yes_Rsvp_Amount: 5| Event_Name: WWD Meetup September} ']

2019-08-01 | ['{Yes_Rsvp_Amount: 38| Event_Name: Introduction to Deep Learning} ', '{Yes_Rsvp_Amount: 29| Event_Name: React Zurich August Meetup} ', '{Yes_Rsvp_Amount: 27| Event_Name: Cloud Native Computing Meetup | August 2019} ', '{Yes_Rsvp_Amount: 22| Event_Name: Presentation: Accessible Data Visualization by Luc Guillemot & Tania Boa} ', '{Yes_Rsvp_Amount: 22| Event_Name: Skalierte Agilität Teil 2 - Praxisbesuch bei der SBB} ']

2019-07-01 | ['{Yes_Rsvp_Amount: 68| Event_Name: Future of Mobility - Powered by Blockchain and Air-Mobility Infrastructure } ', '{Yes_Rsvp_Amount: 67| Event_Name: Meetup #3: Branding and UX für Voice User Interfaces} ', '{Yes_Rsvp_Amount: 56| Event_Name: UX Meetup Juli - UX im Novartis Institutes for BioMedical Research} ', '{Yes_Rsvp_Amount: 50| Event_Name: Erfahrungsbericht Kanban in Scrum anwenden - Eine Erfolgsstory} ', '{Yes_Rs

# Hot Cities: Ranking of the cities with the highest number of events

In [45]:
def get_hot_locations(tx):
    for record in tx.run("match(e:Event) where not (e.city='') with "+dateAggregation+" as date, e.city as city, count(*) as amount_events_in_city order by date desc, amount_events_in_city desc  return  date, collect('Events_Amount: '+amount_events_in_city +' | City: '+ city+' ')[.."+amountOfReturnValues+"] as events_in_cities  order by date desc"):
        print(str(record["date"]) + " | " + str(record["events_in_cities"]))

with driver.session() as session:
    session.read_transaction(get_hot_locations)

2019-09-01 | ['Events_Amount: 1 | City: Wallisellen / Zürich ']
2019-08-01 | ['Events_Amount: 7 | City: Zürich ', 'Events_Amount: 3 | City: Bern ']
2019-07-01 | ['Events_Amount: 21 | City: Zürich ', 'Events_Amount: 7 | City: Stuttgart ']
2019-06-01 | ['Events_Amount: 44 | City: Zürich ', 'Events_Amount: 13 | City: Bern ']
2019-05-01 | ['Events_Amount: 1 | City: Zürich ']
2019-04-01 | ['Events_Amount: 1 | City: 8005 Zurich ']
2019-01-01 | ['Events_Amount: 1 | City: 8005 Zurich ']
2018-12-01 | ['Events_Amount: 1 | City: Genève ']
2018-09-01 | ['Events_Amount: 1 | City: 8005 Zurich ']
2018-05-01 | ['Events_Amount: 1 | City: 8005 Zurich ']


# Hot Addresses: Ranking of the addresses with the highest number of events

In [46]:
def get_hot_addresses(tx):
    for record in tx.run("match (e:Event) where not e.address = '' and  not e.city='' with   "+dateAggregation+" as date, e.address as address, e.city as city, count(*) as amount_events_at_address order by  amount_events_at_address desc return date, collect('Events_Amount: '+amount_events_at_address +'| Address: '+ city+' '+'|  '+ address+' ')[.."+amountOfReturnValues+"] as events_at_addresses order by date desc"):
        print(str(record["date"]) + str(record["events_at_addresses"]))

with driver.session() as session:
    session.read_transaction(get_hot_addresses)
    

2019-09-01['Events_Amount: 1| Address: Wallisellen / Zürich |  Richtistrasse 7 ']
2019-08-01['Events_Amount: 4| Address: Zürich |  Limmatstrasse 50 ', 'Events_Amount: 3| Address: Vaduz |  Fabrikweg ']
2019-07-01['Events_Amount: 2| Address: Zürich |  Stüssihofstatt 4 ', 'Events_Amount: 2| Address: Zug |  Dammstrasse 16 ']
2019-06-01['Events_Amount: 4| Address: Zürich |  Vulkanstrasse 106 ', 'Events_Amount: 3| Address: Zürich |  Limmatstrasse 183 ']
2019-05-01['Events_Amount: 1| Address: Zürich |  Hardstrasse 201 ']
2019-04-01['Events_Amount: 1| Address: 8005 Zurich |   Hardstrasse 201 ']
2019-01-01['Events_Amount: 1| Address: 8005 Zurich |   Hardstrasse 201 ']
2018-12-01['Events_Amount: 1| Address: Genève |  Route de Pré-Bois 29 ']
2018-09-01['Events_Amount: 1| Address: 8005 Zurich |   Hardstrasse 201 ']
2018-05-01['Events_Amount: 1| Address: 8005 Zurich |   Hardstrasse 201 ']


# Hot Topics: Ranking of the topics with the highest number of events

In [47]:
def get_hot_topics(tx):
    for record in tx.run("match(e:Event)<-[:ORGANIZES]-(g:Group)-[:IS_ABOUT]->(t:Topic) with "+dateAggregation+" as date, t.urlkey as topic, count(e) as amount_events_per_topic order by amount_events_per_topic desc return date, collect('Events_Amount '+amount_events_per_topic+ ' | Topic: '+topic )[.."+amountOfReturnValues+"] as events_per_topics order by date desc"):
        print(str(record["date"]) + " | " + str(record["events_per_topics"]) )

with driver.session() as session:
    session.read_transaction(get_hot_topics)

2019-09-01 | ['Events_Amount 1 | Topic: paas', 'Events_Amount 1 | Topic: cloudfoundry']
2019-08-01 | ['Events_Amount 6 | Topic: web-development', 'Events_Amount 5 | Topic: computer-programming']
2019-07-01 | ['Events_Amount 14 | Topic: web-development', 'Events_Amount 11 | Topic: newtech']
2019-06-01 | ['Events_Amount 22 | Topic: cryptocurrency', 'Events_Amount 21 | Topic: newtech']
2019-05-01 | ['Events_Amount 1 | Topic: predictive-analytics', 'Events_Amount 1 | Topic: open-data']
2019-04-01 | ['Events_Amount 2 | Topic: devops', 'Events_Amount 2 | Topic: machine-learning']
2019-01-01 | ['Events_Amount 2 | Topic: docker', 'Events_Amount 2 | Topic: devops']
2018-12-01 | ['Events_Amount 2 | Topic: smart-contracts', 'Events_Amount 2 | Topic: paas']
2018-09-01 | ['Events_Amount 2 | Topic: blockchain', 'Events_Amount 2 | Topic: iaas-infrastructure-as-a-service']
2018-05-01 | ['Events_Amount 2 | Topic: blockchain', 'Events_Amount 2 | Topic: iaas-infrastructure-as-a-service']


# Hot Topics: Ranking of the topics and locations with the highest number of events

In [48]:
def get_hot_topics_and_addresses(tx):
    for record in tx.run("match (e:Event)<-[:ORGANIZES]-(g:Group)-[:IS_ABOUT]->(t:Topic) where not e.address = '' and  not e.city='' with  "+dateAggregation+" as date, e.address as address, e.city as city, t.urlkey as topic, count(*) as amount_events_per_topic_location order by  amount_events_per_topic_location desc return date, collect('Events_Amount: '+amount_events_per_topic_location +' | Address: '+ city+' '+'|  '+ address+' '+'|  '+ topic+' ')[.."+amountOfReturnValues+"] as events_per_topics_and_locations order by date desc"):
        print(str(record["date"]) + str(record["events_per_topics_and_locations"]))

with driver.session() as session:
    session.read_transaction(get_hot_topics_and_addresses)

2019-09-01['Events_Amount: 1 | Address: Wallisellen / Zürich |  Richtistrasse 7 |  kubernetes ', 'Events_Amount: 1 | Address: Wallisellen / Zürich |  Richtistrasse 7 |  software-architecture ']
2019-08-01['Events_Amount: 3 | Address: Vaduz |  Fabrikweg |  ethical-hacking ', 'Events_Amount: 3 | Address: Vaduz |  Fabrikweg |  hardware-hacking-and-reverse-engineering ']
2019-07-01['Events_Amount: 2 | Address: Zürich |  Limmatstrasse 183 |  witi ', 'Events_Amount: 2 | Address: Zürich |  Limmatstrasse 183 |  learning ']
2019-06-01['Events_Amount: 4 | Address: Zürich |  Vulkanstrasse 106 |  cloud-computing ', 'Events_Amount: 3 | Address: 8005 Zurich |   Hardstrasse 201 |  big-data-analytics ']
2019-05-01['Events_Amount: 1 | Address: Zürich |  Hardstrasse 201 |  machine-learning ', 'Events_Amount: 1 | Address: Zürich |  Hardstrasse 201 |  data-visualization ']
2019-04-01['Events_Amount: 2 | Address: 8005 Zurich |   Hardstrasse 201 |  blockchain ', 'Events_Amount: 2 | Address: 8005 Zurich |   

# Ranking of the groups that organizes the most events
Ranking is based on amount of Events of the groups

In [52]:
def get_group_ranking(tx):
    for record in tx.run("match (g:Group)-[:ORGANIZES]->(e:Event)  return g.name as Group, count(e) as amountOfEvents order by amountOfEvents desc"):
        print(str(record["amountOfEvents"])  + "\t|" + str(record["Group"]) )

with driver.session() as session:
    session.read_transaction(get_group_ranking)

13	|Code4Fun
6	|CV Labs - Switzerland
5	|OpenTechSchool Zurich
5	|Cybersecurity Liechtenstein
3	|Bitcoin Meetup Switzerland
3	|Big Data Developers in Switzerland
3	|Digitale Initiativen
2	|Agile & Beyond PostFinance
2	|Westhive Innovation Ecosystem
2	|DEFCON Switzerland (DC4131)
2	|Azure Blockchain Zürich
2	|Data Visualization Zürich
2	|WordPress Neuchâtel
2	|Smart City Lab Basel
2	|Voice Meetup Switzerland
2	|Pantalks - tech & non-tech talks @ Colab Zürich
2	|Microservices Zürich
2	|WintiWebDev Talks
2	|DevOps Meetup Zürich
2	|WebDev Chur
2	|SharePoint & Office 365 Community Schweiz
2	|innolab smart mobility
2	|agile Mindset, Zürich
2	|Lean Change Management Meetup
2	|Trust Square - Blockchain Swiss Made
1	|Scrum Skalierung mit Nexus
1	|GDG Zürich
1	|Azure Meetup Konstanz & Region
1	|Swiss Perl Community Meetup
1	|Women Techmakers Switzerland
1	|Complexity and Cynefin Basel
1	|Angular Zürich
1	|<vue-zurich/>
1	|Embedded GNU/Linux Developer
1	|Framer Zurich
1	|PyData Zurich
1	|Stuttgar

# close neo4j connection

In [ ]:
graph.close()